In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS

In [2]:
factor_returns_df = pd.read_pickle('database\monthly_factor_returns_26Oct20.pickle')
master_df_final = pd.read_pickle('database\master_df_final_26Oct20.pickle')
nifty_200 = pd.read_csv('database\ind_nifty200list.csv')

In [3]:
start_date = min(factor_returns_df.Date)

In [4]:
master_df_final.rename(columns={'formatted_date':'Date'},inplace=True)

In [6]:
look_back_window = 12 
Market_exp_df = pd.DataFrame()
three_mreturn_exp = pd.DataFrame()
Profitability_exp = pd.DataFrame()
BookToPrice_exp = pd.DataFrame()
Liquidity_exp = pd.DataFrame()
LogMcap_exp = pd.DataFrame()
for stock in np.unique(master_df_final.Symbol):
#     print("fetching data for "+stock)
    try:

        #Calculting Beta
        # split dependent and independent variable
        X = factor_returns_df.set_index(['Date'])
        y = master_df_final.loc[(master_df_final.Symbol == stock) & (master_df_final.Date>= start_date),:].set_index(['Date'])['1mreturn']

        # Add a constant to the independent value
        X1 = sm.add_constant(X)

        # make regression model 
        model = RollingOLS(y, X1, window=look_back_window)

        # fit model and print results
        results = model.fit()
        factors_list = ['Market', '3mreturn', '6mreturn', 'Profitability', 'BookToPrice', 'Liquidity', 'LogMcap']
        current_stock_Market_exp = results.params.dropna()[['Market']].rename(columns={'Market': stock})
        current_stock_3mreturn_exp = results.params.dropna()[['3mreturn']].rename(columns={'3mreturn': stock})
        current_stock_Profitability_exp = results.params.dropna()[['Profitability']].rename(columns={'Profitability': stock})
        current_stock_BookToPrice_exp = results.params.dropna()[['BookToPrice']].rename(columns={'BookToPrice': stock})
        current_stock_Liquidity_exp = results.params.dropna()[['Liquidity']].rename(columns={'Liquidity': stock})
        current_stock_LogMcap_exp = results.params.dropna()[['LogMcap']].rename(columns={'LogMcap': stock})
        Market_exp_df = pd.concat([Market_exp_df, current_stock_Market_exp], axis=1)
        three_mreturn_exp = pd.concat([three_mreturn_exp, current_stock_3mreturn_exp], axis=1)
        Profitability_exp = pd.concat([Profitability_exp, current_stock_Profitability_exp], axis=1)
        BookToPrice_exp = pd.concat([BookToPrice_exp, current_stock_BookToPrice_exp], axis=1)
        Liquidity_exp = pd.concat([Liquidity_exp, current_stock_Liquidity_exp], axis=1)
        LogMcap_exp = pd.concat([LogMcap_exp, current_stock_LogMcap_exp], axis=1)
    except:
        print("Error calculating beta for "+stock)

Error calculating beta for ADANIGAS
Error calculating beta for ADANIGREEN
Error calculating beta for DALBHARAT
Error calculating beta for HDFCAMC
Error calculating beta for IRCTC
Error calculating beta for POLYCAB
Error calculating beta for SBICARD
Error calculating beta for YESBANK


In [7]:
Market_exp_df.to_pickle('database\MarketExposure26Oct20.pickle')
three_mreturn_exp.to_pickle('database\MomentumExposure26Oct20.pickle')
Profitability_exp.to_pickle('database\QualityExposure26Oct20.pickle')
BookToPrice_exp.to_pickle('database\ValueExposure26Oct20.pickle')
Liquidity_exp.to_pickle('database\LiquidityExposure26Oct20.pickle')
LogMcap_exp.to_pickle('database\SizeExposure26Oct20.pickle')